Incorporate this into the cleaning / making the crosswalk
Then overwrite the crosswalk

In [2]:
import boto3
import pandas as pd
import geopandas as gpd

s3 = boto3.client('s3')
bucket_name = "city-planning-entitlements"

In [5]:
county_parcels = pd.read_parquet('parcels.parquet')

In [6]:
county_parcels.head()

,RollYear,ZIPcode,TaxRateArea_CITY,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,SpecificUseType,...,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1
AIN,,,,,,,,,,,,,,,,,,,,,
2004001003,2019,91304-3327,LOS ANGELES,16,2004-001-003,8321 FAUST AVE LOS ANGELES CA 91304,SFR,0101,Residential,Single Family Residence,...,,,FAUST AVE,,LOS ANGELES CA,91304,20112004001003,34.220220,-118.620669,"(34.22021979, -118.62066881)"
2004001004,2019,91304-3327,LOS ANGELES,16,2004-001-004,8313 FAUST AVE LOS ANGELES CA 91304,SFR,0101,Residential,Single Family Residence,...,,,FAUST AVE,,LOS ANGELES CA,91304,20062004001004,34.220039,-118.620668,"(34.2200391, -118.6206683)"
2004001005,2019,91304-3327,LOS ANGELES,16,2004-001-005,8309 FAUST AVE LOS ANGELES CA 91304,SFR,0100,Residential,Single Family Residence,...,,,FAUST AVE,,LOS ANGELES CA,91304,20092004001005,34.219858,-118.620676,"(34.21985761, -118.62067568)"
2004001008,2019,91304-3332,LOS ANGELES,16,2004-001-008,8325 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,Residential,Single Family Residence,...,,,MAYNARD AVE,,LOS ANGELES CA,91304,20132004001008,34.220334,-118.622706,"(34.22033372, -118.62270552)"
2004001009,2019,91304-3332,LOS ANGELES,16,2004-001-009,8311 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,Residential,Single Family Residence,...,,,MAYNARD AVE,,LOS ANGELES CA,91304,20092004001009,34.220323,-118.623050,"(34.22032263, -118.62304963)"


In [3]:
file_name = "test_crosswalk_parcels_tracts"
s3.download_file(bucket_name, f'data/{file_name}.parquet', f'./data/{file_name}.parquet')
gdf = gpd.read_parquet(f'./data/{file_name}.parquet')

In [4]:
gdf.head()

,AIN,GEOID,geometry
0,2004001003,06037113232,POINT (6374037.180 1903103.568)
1,2004001004,06037113232,POINT (6374036.928 1903037.812)
2,2004001005,06037113232,POINT (6374034.289 1902971.779)
3,2004001008,06037113232,POINT (6373421.743 1903148.836)
4,2004001009,06037113232,POINT (6373317.694 1903145.445)


In [6]:
# Figure out which ones have duplicate geoms
gdf = gdf.assign(
    x = gdf.geometry.centroid.x, 
    y = gdf.geometry.centroid.y, 
)

In [7]:
duplicate_geom = (gdf.groupby(['x', 'y'])
                  .agg({'AIN':'count'})
                  .reset_index()
                  .rename(columns = {'AIN':'num_AIN'})
                 )

In [8]:
duplicate_geom.num_AIN.value_counts()

1      2115905
2         9257
3         2205
4         2030
6         1395
        ...   
207          1
205          1
203          1
197          1
614          1
Name: num_AIN, Length: 254, dtype: int64

In [ ]:
gdf2 = pd.merge(gdf, duplicate_geom, 
                on = ['x', 'y'], how = 'left', validate = 'm:1')

In [ ]:
"""
Redo parts of src/A3
TOC parcels that planning sent over...it's likely that it's tagging one AIN, but not historical ones
Will need to do a check across all the ones that are duplicates with same x, y
Tag the TOC Tier it belongs to
Our crosswalk_parcels_tracts should include a lot of AINs, 
but still need to store num_AIN, as well as total_AIN (which counts AIN after duplicates are dropped)

Are we keeping parcel polygons?
To keep it simple, we can always use centroids, and we only use counts for AIN, and only tag dups
Instead of toc_area, which requires us to have parcelsqft
"""